### Step 0: Load Packages and Libraries

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import os 
from scipy.sparse import coo_matrix

### Step 1: Data Loading and Preprocessing

In [9]:
train_data=pd.read_excel('RDWEB_1000043_HOMEDEPOT_83815_07-03-2024.XLSX')
train_data.columns

Index(['ID', 'SITE', 'ZIP', 'DATE', 'IMAGELINK', 'PRICE', 'PRICE_MULT',
       'EPRICE', 'EINDICATOR', 'ALTPRICE', 'ALTPRICEMULT', 'EALTPRICE',
       'ALTINDICATOR', 'PACK', 'UNIT', 'UOM', 'INVENTORY', 'PRODUCT_ID', 'UPC',
       'COLLECTION_TYPE', 'CATEGORY', 'DESCRIPTION', 'BRAND', 'RATING',
       'NUMBER_OF_REVIEWS', 'PAGEIMAGE', 'MODEL_NUMBER', 'ORIG_UPC',
       'JOB_NUMBER', 'SALE_ENDDATE'],
      dtype='object')

In [10]:
train_data.head()

,ID,SITE,ZIP,DATE,IMAGELINK,PRICE,PRICE_MULT,EPRICE,EINDICATOR,ALTPRICE,...,CATEGORY,DESCRIPTION,BRAND,RATING,NUMBER_OF_REVIEWS,PAGEIMAGE,MODEL_NUMBER,ORIG_UPC,JOB_NUMBER,SALE_ENDDATE
0,1,https://www.homedepot.com,83815,07/03/2024,https://images.thdstatic.com/productImages/6e7...,$239.0,1.0,239.00,NaN,NaN,...,Appliances>Floor Care>Floor Scrubbers & Buffers,Streamline Corded Wet/Dry Hard Floor Cleaner a...,HOOVER,4.4,165.0,https://www.homedepot.com/p/HOOVER-Streamline-...,FH46020V,73502052235,1000043,NaN
1,2,https://www.homedepot.com,83815,07/03/2024,https://images.thdstatic.com/productImages/da9...,$29.98,1.0,29.98,NaN,NaN,...,Appliances>Appliance Parts>Vacuum Parts>Vacuum...,"HydroVac Anti-Microbial Brushroll, 2-Pack",Shark,5.0,2.0,https://www.homedepot.com/p/Shark-HydroVac-Ant...,WDB2,622356597388,1000043,NaN
2,3,https://www.homedepot.com,83815,07/03/2024,https://images.thdstatic.com/productImages/5c7...,$28.98,1.0,28.98,NaN,NaN,...,Appliances>Appliance Parts>Stove Parts,Universal Chrome Drip Bowl for Electric Ranges...,Everbilt,4.2,522.0,https://www.homedepot.com/p/Everbilt-Universal...,98230,91712026112,1000043,NaN
3,4,https://www.homedepot.com,83815,07/03/2024,https://images.thdstatic.com/productImages/d48...,$28.98,1.0,28.98,NaN,NaN,...,Appliances>Appliance Parts>Stove Parts,Chrome Drip Bowl for GE Electric Ranges (4-Pac...,Everbilt,4.2,514.0,https://www.homedepot.com/p/Everbilt-Chrome-Dr...,98231,91712026129,1000043,NaN
4,5,https://www.homedepot.com,83815,07/03/2024,https://images.thdstatic.com/productImages/eca...,$145.0,1.0,145.00,NaN,NaN,...,Appliances>Floor Care>Steam Mops & Steam Cleaners,915e Multi-Purpose On-Demand Steam Cleaner and...,Wagner,4.0,524.0,https://www.homedepot.com/p/Wagner-915e-Multi-...,0282014,24964195343,1000043,NaN


In [11]:
train_data['PRODUCT_ID']

0        327163212
1        323187219
2        307638950
3        307638937
4        202719943
           ...    
37321    320773266
37322    320773261
37323    320773277
37324    320773639
37325    204284494
Name: PRODUCT_ID, Length: 37326, dtype: int64

In [12]:
train_data=train_data[['ID','IMAGELINK', 'PRICE','PRODUCT_ID', 'UPC','CATEGORY', 'DESCRIPTION', 'BRAND', 'RATING',
       'NUMBER_OF_REVIEWS']]
train_data.head(2)

,ID,IMAGELINK,PRICE,PRODUCT_ID,UPC,CATEGORY,DESCRIPTION,BRAND,RATING,NUMBER_OF_REVIEWS
0,1,https://images.thdstatic.com/productImages/6e7...,$239.0,327163212,7350205223,Appliances>Floor Care>Floor Scrubbers & Buffers,Streamline Corded Wet/Dry Hard Floor Cleaner a...,HOOVER,4.4,165.0
1,2,https://images.thdstatic.com/productImages/da9...,$29.98,323187219,62235659738,Appliances>Appliance Parts>Vacuum Parts>Vacuum...,"HydroVac Anti-Microbial Brushroll, 2-Pack",Shark,5.0,2.0


### Basic Operations

In [13]:
train_data.shape

(37326, 10)

In [14]:
train_data.describe()

,ID,PRODUCT_ID,UPC,RATING,NUMBER_OF_REVIEWS
count,37326.00000,3.732600e+04,3.732600e+04,33358.000000,33358.000000
mean,18663.50000,2.518284e+08,5.792844e+10,4.387487,764.223365
std,10775.23241,7.206988e+07,1.125175e+11,0.455855,1904.952240
min,1.00000,1.000000e+08,3.463456e+07,1.000000,1.000000
25%,9332.25000,2.036782e+08,4.348111e+09,4.300000,52.000000
50%,18663.50000,3.008627e+08,1.945551e+10,4.500000,192.000000
75%,27994.75000,3.154734e+08,8.439820e+10,4.700000,659.000000
max,37326.00000,3.314683e+08,9.506000e+11,5.000000,34881.000000


In [15]:
train_data.isnull().sum()

ID                      0
IMAGELINK               0
PRICE                   1
PRODUCT_ID              0
UPC                     0
CATEGORY              797
DESCRIPTION             0
BRAND                   6
RATING               3968
NUMBER_OF_REVIEWS    3968
dtype: int64

In [16]:
train_data['RATING'].fillna(0,inplace=True)
train_data['NUMBER_OF_REVIEWS'].fillna(0,inplace=True)

In [17]:
train_data.isnull().sum()

ID                     0
IMAGELINK              0
PRICE                  1
PRODUCT_ID             0
UPC                    0
CATEGORY             797
DESCRIPTION            0
BRAND                  6
RATING                 0
NUMBER_OF_REVIEWS      0
dtype: int64

In [18]:
train_data.dropna(inplace=True)

In [19]:
train_data.isnull().sum()

ID                   0
IMAGELINK            0
PRICE                0
PRODUCT_ID           0
UPC                  0
CATEGORY             0
DESCRIPTION          0
BRAND                0
RATING               0
NUMBER_OF_REVIEWS    0
dtype: int64

In [20]:
train_data.shape

(36522, 10)

In [21]:
train_data.columns

Index(['ID', 'IMAGELINK', 'PRICE', 'PRODUCT_ID', 'UPC', 'CATEGORY',
       'DESCRIPTION', 'BRAND', 'RATING', 'NUMBER_OF_REVIEWS'],
      dtype='object')

In [22]:
train_data.head(5)

,ID,IMAGELINK,PRICE,PRODUCT_ID,UPC,CATEGORY,DESCRIPTION,BRAND,RATING,NUMBER_OF_REVIEWS
0,1,https://images.thdstatic.com/productImages/6e7...,$239.0,327163212,7350205223,Appliances>Floor Care>Floor Scrubbers & Buffers,Streamline Corded Wet/Dry Hard Floor Cleaner a...,HOOVER,4.4,165.0
1,2,https://images.thdstatic.com/productImages/da9...,$29.98,323187219,62235659738,Appliances>Appliance Parts>Vacuum Parts>Vacuum...,"HydroVac Anti-Microbial Brushroll, 2-Pack",Shark,5.0,2.0
2,3,https://images.thdstatic.com/productImages/5c7...,$28.98,307638950,9171202611,Appliances>Appliance Parts>Stove Parts,Universal Chrome Drip Bowl for Electric Ranges...,Everbilt,4.2,522.0
3,4,https://images.thdstatic.com/productImages/d48...,$28.98,307638937,9171202612,Appliances>Appliance Parts>Stove Parts,Chrome Drip Bowl for GE Electric Ranges (4-Pac...,Everbilt,4.2,514.0
4,5,https://images.thdstatic.com/productImages/eca...,$145.0,202719943,2496419534,Appliances>Floor Care>Steam Mops & Steam Cleaners,915e Multi-Purpose On-Demand Steam Cleaner and...,Wagner,4.0,524.0


### Step 2: EDA (Exploratory Data Analysis)

In [25]:
num_items=train_data['PRODUCT_ID'].nunique()
num_ratings=train_data['RATING'].nunique()
print(f'Number of unique items: {num_items}')
print(f'Number of unique ratings: {num_ratings}')

Number of unique items: 36522
Number of unique ratings: 40


### Step 3: Data Cleaning and Tag Creations

In [26]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS 

nlp=spacy.load('en_core_web_sm')

def clean_and_extract_tags(text):
    doc=nlp(text.lower())
    tags=[token.text for token in doc if token.text.isalnum() and token.text not in STOP_WORDS]
    return ', '.join(tags)

columns_to_extract_tags_from=['CATEGORY','BRAND','DESCRIPTION']

for column in columns_to_extract_tags_from:
    train_data[column] = train_data[column].apply(clean_and_extract_tags)

In [27]:
train_data.head()

,ID,IMAGELINK,PRICE,PRODUCT_ID,UPC,CATEGORY,DESCRIPTION,BRAND,RATING,NUMBER_OF_REVIEWS
0,1,https://images.thdstatic.com/productImages/6e7...,$239.0,327163212,7350205223,"appliances, floor, care, floor, scrubbers, buf...","streamline, corded, wet, dry, hard, floor, cle...",hoover,4.4,165.0
1,2,https://images.thdstatic.com/productImages/da9...,$29.98,323187219,62235659738,"appliances, appliance, parts, vacuum, parts, v...","hydrovac, anti, microbial, brushroll, 2, pack",shark,5.0,2.0
2,3,https://images.thdstatic.com/productImages/5c7...,$28.98,307638950,9171202611,"appliances, appliance, parts, stove, parts","universal, chrome, drip, bowl, electric, range...",everbilt,4.2,522.0
3,4,https://images.thdstatic.com/productImages/d48...,$28.98,307638937,9171202612,"appliances, appliance, parts, stove, parts","chrome, drip, bowl, ge, electric, ranges, 4",everbilt,4.2,514.0
4,5,https://images.thdstatic.com/productImages/eca...,$145.0,202719943,2496419534,"appliances, floor, care, steam, mops, steam, c...","915e, multi, purpose, demand, steam, cleaner, ...",wagner,4.0,524.0


In [28]:
train_data['Tags']=train_data[columns_to_extract_tags_from].apply(lambda row:', '.join(row),axis=1)

### Rating Base Recommendation System

In [48]:
train_data.head(2)

,ID,IMAGELINK,PRICE,PRODUCT_ID,UPC,CATEGORY,DESCRIPTION,BRAND,RATING,NUMBER_OF_REVIEWS,Tags
0,1,https://images.thdstatic.com/productImages/6e7...,$239.0,327163212,7350205223,"appliances, floor, care, floor, scrubbers, buf...","streamline, corded, wet, dry, hard, floor, cle...",hoover,4.4,165.0,"appliances, floor, care, floor, scrubbers, buf..."
1,2,https://images.thdstatic.com/productImages/da9...,$29.98,323187219,62235659738,"appliances, appliance, parts, vacuum, parts, v...","hydrovac, anti, microbial, brushroll, 2, pack",shark,5.0,2.0,"appliances, appliance, parts, vacuum, parts, v..."


In [51]:
average_ratings=train_data.groupby(['DESCRIPTION','NUMBER_OF_REVIEWS','BRAND','IMAGELINK'])['RATING'].mean().reset_index()
average_ratings

,DESCRIPTION,NUMBER_OF_REVIEWS,BRAND,IMAGELINK,RATING
0,"0, 11, gph, adjustable, dripper, spike, 180, d...",358.0,dig,https://images.thdstatic.com/productImages/142...,4.5
1,"0, 14, gph, 360, adjustable, drip, emitter, st...",358.0,dig,https://images.thdstatic.com/productImages/e8f...,4.5
2,"0, 14, gph, 360, adjustable, drip, emitter, st...",358.0,dig,https://images.thdstatic.com/productImages/0e9...,4.5
3,"0, 14, gph, adjustable, multi, stream, emitter...",469.0,dig,https://images.thdstatic.com/productImages/801...,4.5
4,"0, 15, psi, increment, compressed, air, test, ...",98.0,danco,https://images.thdstatic.com/productImages/7cf...,3.4
...,...,...,...,...,...
36510,"zinsser, 1, gallon, white, interior, water, ba...",298.0,zinsser,https://images.thdstatic.com/productImages/4d2...,4.3
36511,"zipnet, camo, adjustable, cargo, net",25.0,keeper,https://images.thdstatic.com/productImages/3c6...,4.0
36512,"zp4, contains, 4, 10, ft, steel, spring, loade...",293.0,zipwall,https://images.thdstatic.com/productImages/d60...,4.5
36513,"½, barbed, tee, pack, x, tee",63.0,dig,https://images.thdstatic.com/productImages/994...,4.4


In [50]:
top_rated_items=average_ratings.sort_values(by='RATING',ascending=False)
top_rated_items

,DESCRIPTION,NUMBER_OF_REVIEWS,BRAND,IMAGELINK,RATING
19159,"black, padded, leather, waist, tool, belt",2.0,husky,https://images.thdstatic.com/productImages/a78...,5.0
16738,"8, oz, designer, collection, tranquil, gray, s...",2.0,"behr, premium, plus",https://images.thdstatic.com/productImages/1ba...,5.0
10729,"30, white, regula, duty, vertical, rail, shelf...",1.0,everbilt,https://images.thdstatic.com/productImages/71a...,5.0
3818,"10, nickel, plated, steel, finishing, washers,...",1.0,everbilt,https://images.thdstatic.com/productImages/18a...,5.0
18834,"battery, operated, bright, white, 20, lumens, ...",3.0,"private, brand, unbranded",https://images.thdstatic.com/productImages/a12...,5.0
...,...,...,...,...,...
1664,"1, plastic, pole, end, caps, matte, black",0.0,everbilt,https://images.thdstatic.com/productImages/982...,0.0
1663,"1, plastic, pex, pipe, j, hook, pipe, support,...",0.0,apollo,https://images.thdstatic.com/productImages/41e...,0.0
5411,"12, phillips, flat, head, sheet, metal, screws...",0.0,everbilt,https://images.thdstatic.com/productImages/44e...,0.0
25338,"m4, mm, stainless, steel, pan, head, phillips,...",0.0,everbilt,https://images.thdstatic.com/productImages/998...,0.0


In [32]:
rating_base_recommendation=top_rated_items.head(10)

In [52]:
rating_base_recommendation['RATING']=rating_base_recommendation['RATING'].astype(int)
rating_base_recommendation['NUMBER_OF_REVIEWS']=rating_base_recommendation['NUMBER_OF_REVIEWS'].astype(int)

C:\Users\PushkarJaiswal\AppData\Local\Temp\ipykernel_1692\1013428174.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_base_recommendation['RATING']=rating_base_recommendation['RATING'].astype(int)
C:\Users\PushkarJaiswal\AppData\Local\Temp\ipykernel_1692\1013428174.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_base_recommendation['NUMBER_OF_REVIEWS']=rating_base_recommendation['NUMBER_OF_REVIEWS'].astype(int)


In [40]:
rating_base_recommendation

,DESCRIPTION,NUMBER_OF_REVIEWS,BRAND,IMAGELINK,RATING
19159,"black, padded, leather, waist, tool, belt",2,husky,https://images.thdstatic.com/productImages/a78...,5
16738,"8, oz, designer, collection, tranquil, gray, s...",2,"behr, premium, plus",https://images.thdstatic.com/productImages/1ba...,5
10729,"30, white, regula, duty, vertical, rail, shelf...",1,everbilt,https://images.thdstatic.com/productImages/71a...,5
3818,"10, nickel, plated, steel, finishing, washers,...",1,everbilt,https://images.thdstatic.com/productImages/18a...,5
18834,"battery, operated, bright, white, 20, lumens, ...",3,"private, brand, unbranded",https://images.thdstatic.com/productImages/a12...,5
13460,"5, gal, professional, strength, industrial, de...",3,ecolab,https://images.thdstatic.com/productImages/a63...,5
3811,"10, mm, zinc, plated, split, lock, washer, 3, ...",1,everbilt,https://images.thdstatic.com/productImages/741...,5
10724,"30, watt, 6, amp, white, usb, type, c, c, 15, ...",1,leviton,https://images.thdstatic.com/productImages/06d...,5
30269,"small, cabo, flair, terra, cotta, clay, pot",2,pennington,https://images.thdstatic.com/productImages/a75...,5
35128,"x, 50, ft, twisted, polypropylene, rope, yello...",4,everbilt,https://images.thdstatic.com/productImages/c81...,5


### Content Base Recommendation System (User Preferences or Items Similarities)

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def content_based_recommendation(train_data,item_name,top_n=10):
    if item_name not in train_data['DESCRIPTION'].values:
        print(f"Item '{item_name}' not found in the training data.")
        return pd.DataFrame()

    tfidf_vectorizer=TfidfVectorizer(stop_words='english')

    tfidf_matrix_content= tfidf_vectorizer.fit_transform(train_data['Tags'])

    cosine_similarities_content = cosine_similarity(tfidf_matrix_content, tfidf_matrix_content)

    item_index = train_data[train_data['DESCRIPTION'] == item_name].index[0]

    similar_items = list(enumerate(cosine_similarities_content[item_index]))

    similar_items = sorted(similar_items, key=lambda x: x[1], reverse=True)

    top_similar_items = similar_items[1:top_n+1]

    recommended_item_indices = [x[0] for x in top_similar_items]

    recommended_items_details = train_data.iloc[recommended_item_indices][['DESCRIPTION', 'NUMBER_OF_REVIEWS', 'BRAND', 'IMAGELINK', 'RATING','PRODUCT_ID']]

    return recommended_items_details

In [54]:
item_name = 'OPI Infinite Shine, Nail Lacquer Nail Polish, Bubble Bath'
content_based_rec = content_based_recommendation(train_data, item_name, top_n=8)

content_based_rec

Item 'OPI Infinite Shine, Nail Lacquer Nail Polish, Bubble Bath' not found in the training data.


""


In [55]:
train_data['DESCRIPTION'][4]

'915e, multi, purpose, demand, steam, cleaner, wallpaper, remover'

In [56]:
item_name = '915e, multi, purpose, demand, steam, cleaner, wallpaper, remover'
content_based_rec = content_based_recommendation(train_data, item_name, top_n=8)

content_based_rec

,DESCRIPTION,NUMBER_OF_REVIEWS,BRAND,IMAGELINK,RATING,PRODUCT_ID
5,"corded, steam, mop, cleaner, hard, floors, whi...",78.0,shark,https://images.thdstatic.com/productImages/199...,4.7,330317520
20305,"1, qt, steam, gray, interior, floor, base, mat...",1385.0,"rust, oleum, home",https://images.thdstatic.com/productImages/e24...,4.2,313143182
20288,"1, qt, steam, gray, interior, floor, base, sem...",1385.0,"rust, oleum, home",https://images.thdstatic.com/productImages/94d...,4.2,313143209
35600,"6, ft, steam, dryer, installation, kit, 90, de...",55.0,everbilt,https://images.thdstatic.com/productImages/6a1...,4.5,312040295
4699,"96, oz, pet, urine, destroyer, odor, remover, ...",192.0,resolve,https://images.thdstatic.com/productImages/76f...,4.2,312609319
4691,"96, oz, carpet, steam, cleaning, oz, ounce",162.0,resolve,https://images.thdstatic.com/productImages/e1a...,4.4,206501137
4692,"96, oz, steam, carpet, cleaning, oz, ounce",113.0,resolve,https://images.thdstatic.com/productImages/9c6...,4.6,207127512
35496,"fht, x, fht, x, 72, braided, stainless, steam,...",285.0,everbilt,https://images.thdstatic.com/productImages/769...,4.6,205854154
